## Logging on

Use Selenium to visit https://webapps1.chicago.gov/buildingrecords/ and accept the agreement.

> Think about when you use `.find_element...` and when you use `.find_elementSSS...`

In [150]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

from webdriver_manager.chrome import ChromeDriverManager

In [151]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/vincentvanbuskirk/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/s0/4l3lgvt927bg9rlncds94m2c0000gn/T/ipykernel_30192/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [152]:
driver.get('https://webapps1.chicago.gov/buildingrecords/')

In [153]:
driver.find_element(By.ID, 'rbnAgreement1').click()

In [154]:
driver.find_element(By.ID, 'submit').click()

## Searching

Search for **400 E 41ST ST**.

In [155]:
driver.find_element(By.ID, 'fullAddress').send_keys('400 E 41ST ST')

In [156]:
driver.find_element(By.ID, 'submit').click()

## Saving tables with pandas

Use pandas to save a CSV of all **permits** to `Permits - 400 E 41ST ST.csv`. Note that there are **different sections of the page**, not just one long permits table.

> - *Tip: When using `.read_html`, try using `flavor='lxml'` and comparing the results to `flavor='html5lib'`. Which works better?*
> - *Tip: You might need to install `html5lib` using `pip`. If so, you'll need to restart the notebook using **Kernel > Restart** before it will work.*

In [157]:

table = driver.find_element(By.ID, 'resultstable_permits')

In [158]:
import pandas as pd

df = pd.read_html(table.get_attribute('outerHTML'), flavor='html5lib')[0]
df.head()

,PERMIT #,DATE ISSUED,DESCRIPTION OF WORK
0,100845718,10/11/2019,ERECT TWO SCAFFOLDS FROM 10/14/2019 TO 10/14/2...
1,100778302,08/17/2018,PERMIT EXPIRES ON 10/17/2018 Erection Starts: ...
2,100721255,08/24/2017,PERMIT EXPIRES ON 10/24/2017 ERECTION STARTS: ...
3,100693399,03/03/2017,INSTALLATION OF LOW VOLTAGE BURGLAR ALARM INTE...
4,100665436,08/24/2016,PERMIT EXPIRES ON 10/24/2016 ERECTION STARTS: ...


In [14]:
df.to_csv('Permits - 400 E 41ST ST.csv', index=False)

## Saving tables the long way

Save a CSV of all DOB inspections to `Inspections - 400 E 41ST ST.csv`.

This is more complicated than the last one becuse **we also need to save the URL to the inspection** (see how the inspection number is a link?). As a result, you won't be able to use pandas! Instead, you'll need to use a loop and create a list of dictionaries.

You can use Selenium or you can feed the source to BeautifulSoup. You should have approximately 160 rows.

You'll probably need to find the table first, then the rows inside, then the cells inside of each row. You'll probably use lots of list indexing. I might recommend XPath for finding the table.

*Tip: If you get a "list index out of range" error, it's probably due to an issue involving `thead` vs `tbody` elements. What are they? What are they for? What's in them? There are a few ways to troubleshoot it.*

In [159]:
from bs4 import BeautifulSoup

doc = BeautifulSoup(driver.page_source)

In [160]:
inspection_table = doc.find(id='resultstable_inspections')

In [161]:
all_rows = []
for row in inspection_table.find('tbody').find_all('tr'):
    row_dic = {}
    all_td = row.find_all('td')
    
    row_dic['inspection_num'] = all_td[0].a.text
    row_dic['inspection_url'] = f"https://webapps1.chicago.gov{all_td[0].a['href']}"
    row_dic['inspection_date'] = all_td[1].span.text
    row_dic['status'] = all_td[2].text
    row_dic['type_description'] = all_td[3].text
    
    all_rows.append(row_dic)

In [162]:
pd.DataFrame(all_rows).to_csv('Inspections - 400 E 41ST ST.csv', index=False)

### Bonus preview of Wednesday's content

**You prrrrrobably shouldn't do this one unless you want a real challenge.**

If you click the inspection number, it'll open up a new window that shows you details of the violations from that visit. Count the number of violations for each visit and save it in a new column called **num_violations**.

Save this file as `Inspections - 400 E 41ST ST - with counts.csv`.

Since clicking the link opens in a new window, we have to say "Hey Selenium, pay attention to that new window!" We do that with `driver.switch_to.window(driver.window_handles[-1])` (each window gets a `window_handle`, and we're just asking the driver to switch to the last one.). A rough sketch of what your code will look like is here:

```python
# Click the link that opens the new window

# Switch to the new window/tab
driver.switch_to.window(driver.window_handles[-1])

# Do your scraping in here

# Close the new window/tab
driver.close()

# Switch back to the original window/tab
driver.switch_to.window(driver.window_handles[0])
```

You'll want to play around with them individually before you try it with the whole set - the ones that pass are very different pages than the ones with violations! There are a few ways to get the number of violations, some easier than others.

In [163]:
table = driver.find_element(By.ID, 'resultstable_inspections')

In [164]:
inspection_violation_list = []
for row in table.find_elements(By.TAG_NAME, 'tr')[1:]:
    row.find_element(By.TAG_NAME, 'a').click()
    driver.switch_to.window(driver.window_handles[-1])
    try:
        inspection_violations = driver.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')
        inspection_violation_list.append(len(inspection_violations))
    except:
        inspection_violation_list.append(0)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

In [165]:
df = pd.read_csv('Inspections - 400 E 41ST ST.csv')

In [167]:
df['inspection_num'] = inspection_violation_list
df.head()

,inspection_num,inspection_url,inspection_date,status,type_description
0,2,https://webapps1.chicago.gov/buildingrecords/i...,20211025,FAILED,ANNUAL INSPECTION
1,4,https://webapps1.chicago.gov/buildingrecords/i...,20210823,FAILED,CONSERVATION ANNUAL
2,2,https://webapps1.chicago.gov/buildingrecords/i...,20210809,FAILED,ANNUAL INSPECTION
3,20,https://webapps1.chicago.gov/buildingrecords/i...,20210303,FAILED,ANNUAL INSPECTION
4,10,https://webapps1.chicago.gov/buildingrecords/i...,20201130,FAILED,ANNUAL INSPECTION


In [168]:
df.to_csv('Inspections - 400 E 41ST ST.csv', index=False)

In [169]:
driver.close()